In [3]:
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import shutil

In [4]:
from spiral_tools.augmentation import filter_brightness,rotation_90,filter_saturation
from spiral_tools.utils import path_sequence,format_line,create_yolo_dataset_format,show_images_figure
from spiral_tools.constants import label_color_dict, label_dict

In [10]:
os.chdir("..")
os.getcwd()

'C:\\Users\\Asus\\Desktop\\SpirAl Lab'

In [11]:
root = "."

ds_path = path_sequence(root,"Dataset_handle","datasets","tagged","detect")
label_file_path = path_sequence(ds_path,"labels")
image_file_path = path_sequence(ds_path,"images")

image_save_path = "./data_augmentation/augmentated/frames"
label_save_path = "./data_augmentation/augmentated/labels"

In [12]:
def rotate_yolo_labels_90(lines, img_width, img_height):
    
    rotated_lines = []

    for line in lines:
        class_idx, x_center_norm, y_center_norm, width_norm, height_norm = map(float, line.split())
        class_idx = int(class_idx)

        # Convert normalized center coordinates to pixel coordinates
        x_center = x_center_norm * img_width
        y_center = y_center_norm * img_height

        # Compute new pixel coordinates for rotated bounding box
        x_new = y_center
        y_new = img_width - x_center

        # Convert new pixel coordinates back to normalized coordinates
        x_new_norm = x_new / img_height
        y_new_norm = y_new / img_width

        # NEW ADDED
        width_new_norm = height_norm
        height_new_norm = width_norm

        # Append rotated annotation to list of strings
        rotated_line = f"{class_idx} {x_new_norm:.6f} {y_new_norm:.6f} {width_new_norm:.6f} {height_new_norm:.6f}\n"
        rotated_lines.append(rotated_line)

    return rotated_lines

In [13]:
def augment_frame(orig_frame,lines):
    frame_brightness,filter_cat = filter_brightness(orig_frame)
    frame_saturation,filter_cat = filter_saturation(orig_frame)   
    augmanted_frames = [frame_brightness,frame_saturation]
    
    augmanted_frame_lines = [lines,lines]

    for frame in [frame_brightness,frame_saturation]:

        for i in range(3):
    
            rotated_frame , rot_cat= rotation_90(frame,i+1)
            
            newlines = []
            if i+1 == 1:
                newlines = rotate_yolo_labels_90(lines,frame.shape[1],frame.shape[0])
            elif i+1 == 2:
                newlines = rotate_yolo_labels_90(lines,frame.shape[1],frame.shape[0])
                newlines = rotate_yolo_labels_90(newlines,frame.shape[1],frame.shape[0])
            elif i+1 == 3:
                newlines = rotate_yolo_labels_90(lines,frame.shape[1],frame.shape[0])
                newlines = rotate_yolo_labels_90(newlines,frame.shape[1],frame.shape[0])
                newlines = rotate_yolo_labels_90(newlines,frame.shape[1],frame.shape[0])
            
            augmanted_frames.append(rotated_frame)
            augmanted_frame_lines.append(newlines)
        
    return augmanted_frames,augmanted_frame_lines



In [14]:
os.getcwd()

'C:\\Users\\Asus\\Desktop\\SpirAl Lab'

In [19]:
show_augmanted_frames = False
save_new_data = True

In [26]:
frame_names = []

font = cv2.FONT_HERSHEY_DUPLEX
for i,c in tqdm(enumerate(["train","test","val"])):
    labels_file = path_sequence(label_file_path,c)
    label_names = os.listdir(labels_file)

    for label_name in label_names:
        
        # etiketler
        f = open(path_sequence(labels_file,label_name),"r")
        lines = f.readlines()
        f.close()

        im =cv2.imread(path_sequence(image_file_path,c,f"{label_name[0:-4]}.jpg"))
        aug_frames, aug_frame_lines = augment_frame(im,lines)
        
        if show_augmanted_frames:
            
            for frame,lines in zip(aug_frames,aug_frame_lines):
                for line in lines:
                    label, norm_center_x, norm_center_y, norm_width, norm_height = format_line(line)

                    width = frame.shape[1] * norm_width
                    height = frame.shape[0] * norm_height
                    x1 = int(frame.shape[1]*norm_center_x) - int(width/2)
                    x2 = int(frame.shape[1]*norm_center_x) + int(width/2)
                    y1 = int(frame.shape[0]*norm_center_y) - int(height/2)
                    y2 = int(frame.shape[0]*norm_center_y) + int(height/2)

                    # etiketleri çizdirelim
                    cv2.putText(frame,f"{label_dict[int(label)]}",(x1+5,y1+15),font,0.55,label_color_dict[int(label)])
                    cv2.rectangle(frame,(x1,y1),(x2,y2),label_color_dict[int(label)],2)

                cv2.imshow("q",cv2.resize(frame,(1200,800)))
                key = cv2.waitKey(0)
                if key == ord("q"):
                    break
                elif key == ord(" "):
                    continue
                    
        if save_new_data:
            counter = 1
            for aug_frame,aug_lines in zip(aug_frames,aug_frame_lines):
                cv2.imwrite(path_sequence(image_save_path,f"{label_name[0:-4]}_aug_{counter}.jpg"),aug_frame)
                f = open(path_sequence(label_save_path,f"{label_name[0:-4]}_aug_{counter}.txt"),"w")
                f.writelines(aug_lines)
                f.close()
                print(path_sequence(image_save_path,f"{label_name[0:-4]}_aug_{counter}.jpg"))
                counter +=1


0it [00:00, ?it/s]

1
./data_augmentation/augmentated/frames/frame_000000_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_7.jpg
./data_augmentation/augmentated/frames/frame_000000_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_000004_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_7.jpg
./data_augmentation/augmentated/frames/frame_000004_aug_8.jpg
1
./

1
./data_augmentation/augmentated/frames/frame_000228_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_7.jpg
./data_augmentation/augmentated/frames/frame_000228_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_000236_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_7.jpg
./data_augmentation/augmentated/frames/frame_000236_aug_8.jpg
1
./

./data_augmentation/augmentated/frames/frame_000592_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_000608_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_7.jpg
./data_augmentation/augmentated/frames/frame_000608_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_000800_aug_1.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_2.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_3.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_4.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_5.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_6.jpg
./data_augmentation/augmentated/frames/frame_000800_aug_7.jpg
./da

./data_augmentation/augmentated/frames/frame_002624_aug_7.jpg
./data_augmentation/augmentated/frames/frame_002624_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_002700_aug_1.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_2.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_3.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_4.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_5.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_6.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_7.jpg
./data_augmentation/augmentated/frames/frame_002700_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_002728_aug_1.jpg
./data_augmentation/augmentated/frames/frame_002728_aug_2.jpg
./data_augmentation/augmentated/frames/frame_002728_aug_3.jpg
./data_augmentation/augmentated/frames/frame_002728_aug_4.jpg
./data_augmentation/augmentated/frames/frame_002728_aug_5.jpg
./data_augmentation/augmentated/frames/frame_002728_aug_6.jpg
./da

1
./data_augmentation/augmentated/frames/frame_004080_aug_1.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_2.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_3.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_4.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_5.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_6.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_7.jpg
./data_augmentation/augmentated/frames/frame_004080_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_004116_aug_1.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_2.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_3.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_4.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_5.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_6.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_7.jpg
./data_augmentation/augmentated/frames/frame_004116_aug_8.jpg
1
./

1
./data_augmentation/augmentated/frames/frame_010416_aug_1.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_2.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_3.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_4.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_5.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_6.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_7.jpg
./data_augmentation/augmentated/frames/frame_010416_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_010452_aug_1.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_2.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_3.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_4.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_5.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_6.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_7.jpg
./data_augmentation/augmentated/frames/frame_010452_aug_8.jpg
1
./

./data_augmentation/augmentated/frames/frame_011660_aug_7.jpg
./data_augmentation/augmentated/frames/frame_011660_aug_8.jpg
1
./data_augmentation/augmentated/frames/frame_011688_aug_1.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_2.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_3.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_4.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_5.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_6.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_7.jpg
./data_augmentation/augmentated/frames/frame_011688_aug_8.jpg
1
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_64_aug_1.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_64_aug_2.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_64_aug_3.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_64_aug_4.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_64_aug_5.jpg
./data

3it [00:41, 13.76s/it]

./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_95_aug_6.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_95_aug_7.jpg
./data_augmentation/augmentated/frames/TUYZ_2023_Ornek_Video_95_aug_8.jpg
